In [ ]:


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn import tree
data = pd.read_csv('D1_Speed_Profile.csv')
df = pd.DataFrame(data, columns = data.columns)

df.shape
df = df.sample(frac = 1)

x = df.drop(columns = ["Filename","Rating"])
#print(x[0:5])
y = df["Rating"]#save the feature name and target variables
feature_names = x.columns

labels = y.unique()#split the dataset
le = LabelEncoder()
le.fit(y.astype(str))
y = le.transform(y.astype(str))

X_train, test_x, y_train, test_lab = train_test_split(x,y,
                                                 test_size = 0.3,
                                                 random_state = 42)
unique, counts = np.unique(y_train, return_counts=True)
#dict(zip(unique, counts))


In [ ]:
#Create a RF Classifier
clf=RandomForestClassifier(n_estimators=10, 
                            max_depth=3,
                            max_features='auto',
                            bootstrap=True, 
                            n_jobs=-1, 
                            random_state=0)

In [ ]:
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
spare_cn=str(labels)
test_labels= str(y)
print(spare_cn)
print(np.unique(y))

In [ ]:
train_pred = clf.predict(X_train)
feature_imp = pd.Series(clf.feature_importances_,index=feature_names).sort_values(ascending=False)
sns.barplot(x=feature_imp, y=feature_imp.index)
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.rcParams["figure.figsize"] = (7,10)
plt.legend()
plt.show()

for i,v in enumerate(feature_imp):
	print('Feature: %s, Score: %.5f' % (feature_imp.index[i],v))

In [ ]:
!pip install shap
import shap

In [ ]:

X_importance = X_train ######## test set passed, can be passed training set as well

# Explain model predictions using shap library:
explainer = shap.TreeExplainer(clf) ###### lgb_model is the fitted model
shap_values = explainer.shap_values(X_importance)
shap.summary_plot(shap_values, X_importance)

importance_df = pd.DataFrame(index=[X_importance.columns.tolist()])
#importance_df = pd.DataFrame([X_importance.columns.tolist()]).T
#importance_df.columns = ['column_name']
print(importance_df)
for i in np.unique(y):
  shap_sum = np.abs(shap_values[i]).mean(axis=0)
  shap_sum= shap_sum.astype('float')
  importance_df[i] = shap_sum.tolist()
  


importance_df = importance_df.sort_values(0, ascending=False)
result = importance_df.idxmax(axis=1)
importance_df
importance_df.head(5)
print(result)
data_top = importance_df.head(5) 
important_fetures= data_top.index.tolist()